In [16]:
import csv
import os
import time
import pickle as pkl
import numpy as np
from statistics import mean 
from sklearn.metrics import f1_score
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torch_data
import mca
import pandas as pd

In [17]:
def _loadTxt(fName):
    return np.loadtxt(fName, delimiter=',', skiprows=1, dtype=np.float32)

def cache(dirPath):
    cacheF = os.path.join(dirPath, 'cache.pkl')
    if os.path.isfile(cacheF):
        with open(cacheF, 'rb') as f:
            return pkl.load(f)
    return None

def loadBibtex(dirPath):
    cacheF = os.path.join(dirPath, 'cache.pkl')
    d = cache(dirPath)
    if d is None:
        train = _loadTxt(os.path.join(dirPath, "bibtex-train.csv"))
        trainX, trainY = np.split(train, [1836], axis=1)
        test = _loadTxt(os.path.join(dirPath, "bibtex-test.csv"))
        testX, testY = np.split(test, [1836], axis=1)
        d = {'trainX': trainX, 'trainY': trainY,
            'testX': testX, 'testY': testY}
        with open(cacheF, 'wb') as f:
            pkl.dump(d, f)

    return d

In [18]:
data = loadBibtex("bibtex")

trainX = data['trainX']
trainY = data['trainY']
testX = data['testX']
testY = data['testY']

In [20]:
trainX.shape, testX.shape, trainY.shape, testY.shape

((4880, 1836), (2515, 1836), (4880, 159), (2515, 159))

In [21]:
tr_tstX = np.vstack((trainX,testX))
tr_tstY = np.vstack((trainY,testY))
tr_tstX_data = pd.DataFrame(tr_tstX)

In [22]:
mca_ben = mca.MCA(tr_tstX_data, ncols=420)
print(mca.MCA.__doc__)

Run MCA on selected columns of a pd DataFrame.
	
	If the column are specified, assume that they hold
	categorical variables that need to be replaced with
	dummy indicators, otherwise process the DataFrame as is.

	'cols': The columns of the DataFrame to process.
	'ncols': The number of columns before dummy coding. To be passed if cols isn't.
	'benzecri': Perform Benzécri correction (default: True)
	'TOL': value below which to round eigenvalues to zero (default: 1e-4)
	


In [23]:
new_x = mca_ben.fs_r(1)
trainX, testX = np.split(new_x, [4880], axis=0)
trainY, testY = np.split(tr_tstY, [4880], axis=0)

In [24]:
trainX.shape, testX.shape, trainY.shape, testY.shape

((4880, 785), (2515, 785), (4880, 159), (2515, 159))

In [8]:
input_dim = trainX.shape[1]
output_dim = trainY.shape[1]
batch_size_train = trainX.shape[0]//15
batch_size_test = testX.shape[0]//10 

hidden_layer_sizes=[135, 110]

In [9]:
class BibtexData(torch_data.Dataset):
    def __init__(self, X, y):
        super(BibtexData, self).__init__()
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [10]:
train_dset = BibtexData(trainX, trainY) 
test_dset = BibtexData(testX, testY) 

In [11]:
class DenseICNN(nn.Module):
    def __init__(self, input_dim, output_dim,  hidden_layer_sizes, activation='celu', dropout=0.3):
        super(DenseICNN, self).__init__()
        
        
        self.hidden_layer_sizes = hidden_layer_sizes
        self.droput = dropout
        self.activation = activation

        
        self.quadratic_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, output_features, bias=True),
                nn.Dropout(dropout))
            for output_features in hidden_layer_sizes])
        
        sizes = zip(hidden_layer_sizes[:-1], hidden_layer_sizes[1:])
        self.convex_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_features, output_features, bias=True),
                nn.Dropout(dropout))
            for (input_features, output_features) in sizes])
        
        self.final_layer = nn.Linear(hidden_layer_sizes[-1], output_dim, bias=True)
        
    def forward(self, input):
        output = self.quadratic_layers[0](input)
        for quadratic_layer, convex_layer in zip(self.quadratic_layers[1:], self.convex_layers):
            output = convex_layer(output) + quadratic_layer(input)
            if self.activation == 'celu':
                output = torch.celu(output)
        return self.final_layer(output)
    
    def convexify(self):
        for layer in self.convex_layers:
            for sublayer in layer:
                if (isinstance(sublayer, nn.Linear)):
                    sublayer.weight.data.clamp_(0)

In [12]:
ICNN = DenseICNN(input_dim, output_dim,  hidden_layer_sizes, activation='celu') 

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(ICNN.parameters(), lr=0.006, weight_decay=1e-7) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.8)


train_loader = torch_data.DataLoader(train_dset, batch_size=batch_size_train, shuffle=True) 
test_loader = torch_data.DataLoader(test_dset, batch_size=batch_size_test, shuffle=False) 

In [13]:
def train(epochs, ICNN, criterion, optimizer, train_loader, test_loader, scheduler=None, verbose=True, save_dir=None):
    
    for epoch in range(1, epochs+1):
        f1 = []
        ICNN.train()
        for X, y in train_loader:
           
            y_out = ICNN(X)

            loss = criterion(y, torch.sigmoid(y_out))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            ICNN.convexify()  
            
        ICNN.eval()
        for X, y in test_loader:
            
            y_out = ICNN(X)

            test_loss = criterion(y, torch.sigmoid(y_out))
            predY_bin = (torch.sigmoid(y_out).detach().numpy() >= 0.5).astype(np.int)
            trueY_bin = y.detach().numpy()
            f1.append(f1_score(trueY_bin.T, predY_bin.T, average='macro', pos_label=None))
            
        if scheduler is not None:
            scheduler.step()
        freq = max(epochs//20,50)
        if verbose and epoch%freq==0:
            print('Epoch {}/{} || Loss:  Train {:.4f} | Validation {:.4f}'.format(epoch, epochs, loss.item(), test_loss.item()))
            print('F1 score:', mean(f1))

In [14]:
train(500, ICNN, criterion, optimizer, train_loader, test_loader, scheduler)

Epoch 50/500 || Loss:  Train 0.0180 | Validation 0.0063
F1 score: 0.14405609883697534
Epoch 100/500 || Loss:  Train 0.0149 | Validation 0.0049
F1 score: 0.22667493478793732
Epoch 150/500 || Loss:  Train 0.0075 | Validation 0.0056
F1 score: 0.2776935605480138
Epoch 200/500 || Loss:  Train 0.0037 | Validation 0.0040
F1 score: 0.3472302635140728
Epoch 250/500 || Loss:  Train 0.0082 | Validation 0.0040
F1 score: 0.36181467950325535
Epoch 300/500 || Loss:  Train 0.0046 | Validation 0.0046
F1 score: 0.3661285887672208
Epoch 350/500 || Loss:  Train 0.0106 | Validation 0.0045
F1 score: 0.36786566213990207
Epoch 400/500 || Loss:  Train 0.0041 | Validation 0.0052
F1 score: 0.37227686786103836
Epoch 450/500 || Loss:  Train 0.0096 | Validation 0.0047
F1 score: 0.38239835938270456
Epoch 500/500 || Loss:  Train 0.0059 | Validation 0.0044
F1 score: 0.3823312916466281
